In [1]:
import re
import numpy as np
import pandas as pd
import pickle
import chromadb
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.schema.runnable import RunnableMap
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.schema.document import Document
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers import EnsembleRetriever
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings, AzureOpenAIEmbeddings, AzureChatOpenAI, AzureOpenAI
from langchain_mistralai import MistralAIEmbeddings
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_community.llms.vllm import VLLMOpenAI
from langchain_community.llms import Ollama

import urllib.request
import json
from dotenv import load_dotenv
from typing import Any, Dict, Iterator, List, Mapping, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk

import getpass
import os
from operator import itemgetter
import lark
import urllib
import json

from langchain.schema.document import Document
from langchain_community.document_loaders import UnstructuredFileLoader, PyPDFLoader
from tempfile import NamedTemporaryFile
import pypdf
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tempfile
from langchain.chains import LLMChain

chunk_size = 1600 # Demie-page
chunk_overlap = 128

########################################
# Conf file with connexion credentials
import configparser
config = configparser.ConfigParser()
config.read('/Users/aba/Documents/dev/conf.ini')
mixtral_api_base = config['creds']['MIXTRAL_API_BASE']
mixtral_api_key = config['creds']['MIXTRAL_API_KEY']
gpt_api_base = "https://apigatewayinnovation.azure-api.net/openai-api/deployments/gpt-4-32k/chat/completions?api-version=2023-07-01-preview"
gpt_api_key = config['creds']['GPT_API_KEY']

os.environ["MIXTRAL_API_BASE"] = config['creds']['MIXTRAL_API_BASE']
os.environ["MIXTRAL_API_KEY"] = config['creds']['MIXTRAL_API_KEY']
os.environ["GPT_API_KEY"] = config['creds']['GPT_API_KEY']
os.environ["AZURE_OPENAI_API_KEY"] = config['creds']['GPT_API_KEY']
os.environ["OPENAI_API_KEY"] = config['creds']['OPEN_AI_KEY_PERSO']
OPENAI_API_URL ="https://apigatewayinnovation.azure-api.net/openai-api/deployments/gpt-4-32k/chat/completions?api-version=2023-07-01-preview"
#OPENAI_API_URL ="https://apigatewayinnovation.azure-api.net/openai-api/deployments/gpt-35-turbo/chat/completions?api-version=2024-02-01"
OPENAI_API_URL="https://apigatewayinnovation.azure-api.net/openai-api/deployments/gpt-4o/chat/completions?api-version=2024-02-01"

from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator


In [2]:
import os
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults
from langchain.tools import StructuredTool
import requests
import json
import configparser
config = configparser.ConfigParser()
config.read('/Users/aba/Documents/dev/conf.ini')
# set up API key
os.environ["TAVILY_API_KEY"] = config['creds']['TAVILY_API_KEY'] 
from model import get_agent

In [3]:
def call_openai(prompt):
    try:
        url = OPENAI_API_URL

        hdr = {
            # Request headers
            'Content-Type': 'application/json',
            'Cache-Control': 'no-cache',
            'Ocp-Apim-Subscription-Key': os.environ['GPT_API_KEY'],
        }
        data = {
            "messages": [
                {
                    "role": "system",
                    "content": ""
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stop": ["\\n"],
            #"max_tokens" : 10000
        }
        #print(prompt)

        # Request body
        data = json.dumps(data)
        #print(data)
        
        req = urllib.request.Request(url, headers=hdr, data=bytes(data.encode("utf-8")))

        req.get_method = lambda: 'POST'
        response = urllib.request.urlopen(req)
        #print(response.status==200)
        return json.loads(response.read())["choices"][0]["message"]['content']
    except Exception as e:
        print(e)
        return None


class CustomGPT(LLM):
    """
    New definition of the LLM for interacting with LangChain

    """

    # n: int
    # """ parameter on initializing"""

    def _call(
            self,
            prompt: str,
            stop: Optional[List[str]] = None,
            run_manager: Optional[CallbackManagerForLLMRun] = None,
            **kwargs: Any,
    ) -> str:
        """Run the LLM on the given input.

        Args:
            prompt: The prompt to generate from.
            stop: Stop words to use when generating. Model output is cut off at the
                first occurrence of any of the stop substrings.
                If stop tokens are not supported consider raising NotImplementedError.
            run_manager: Callback manager for the run.
            **kwargs: Arbitrary additional keyword arguments. These are usually passed
                to the model provider API call.

        Returns:
            The model output as a string. Actual completions SHOULD NOT include the prompt.
        """
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        return call_openai(prompt) #"Prompt : " + prompt + "\n Anwser : " +  call_openai(prompt)

    def _stream(
            self,
            prompt: str,
            stop: Optional[List[str]] = None,
            run_manager: Optional[CallbackManagerForLLMRun] = None,
            **kwargs: Any,
    ) -> Iterator[GenerationChunk]:
        """Stream the LLM on the given prompt.

        This method should be overridden by subclasses that support streaming.

        If not implemented, the default behavior of calls to stream will be to
        fallback to the non-streaming version of the model and return
        the output as a single chunk.

        Args:
            prompt: The prompt to generate from.
            stop: Stop words to use when generating. Model output is cut off at the
                first occurrence of any of these substrings.
            run_manager: Callback manager for the run.
            **kwargs: Arbitrary additional keyword arguments. These are usually passed
                to the model provider API call.

        Returns:
            An iterator of GenerationChunks.
        """
        for char in call_openai(prompt):
            chunk = GenerationChunk(text=char)
            if run_manager:
                run_manager.on_llm_new_token(chunk.text, chunk=chunk)

            yield chunk

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters."""
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": "GPTModel",
        }

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "customGPT"

    
def instantiate_mixtral() -> VLLMOpenAI:
    """Instantiates the Mixtral LLM."""

    llm = VLLMOpenAI(
        openai_api_key=os.environ["MIXTRAL_API_KEY"],
        openai_api_base=os.environ["MIXTRAL_API_BASE"],
        model_name="mistralai/Mixtral-8X7B-Instruct-v0.1",
        temperature=0.1,
        max_tokens=5000,
        verbose=True,
    )
    return llm



In [4]:
# set up the agent
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)


def get_semrank_result(keyword: str) -> dict:
    """ Search for Semrank result on specific keywrods and parse it"""
    r = requests.post(url="https://semrank.io/admin/api/keywords", data={"query":keyword})
    return json.loads(r.content)


def get_hn_structures(keyword: str) -> dict:
    """Retrieves Hn structures from top 3 website of Semrank's results about specific keyword"""
    results = get_semrank_result(keyword)
    a = {}
    for _, concurrent in results["datas"]["concurrents"].items():
        if concurrent["position"]<=3:
            a[concurrent["url"]] = concurrent["headings"]
            a["title"] = concurrent["title"]
            a["descr"] = concurrent["descr"]
    
    return {"result" : results, "parsed": a}


tool_hn= StructuredTool.from_function(get_hn_structures)


tools = [tavily_tool, tool_hn]

def parse_semrank_object(results, top_concurrent = 3):
    
    # 1. related questions
    related_questions = [i["question"] for _,i in results["datas"]["paa"].items()] 
    other_related_questions = [i["query"] for _,i in results["datas"]['related'].items()]
    
    # 2. Concurrent + docapposte data content => Title, snippet, Content, Hn structure
    list_concurr = results["datas"]['concurrents']
    concurrents_data = []
    positions = sorted([c["position"] for _, c in results["datas"]["concurrents"].items()])
    docaposte_data = []
    all_url = []
    
    ## Data on "concurrents" key
    for _, concurrent in results["datas"]["concurrents"].items():
        a = {}
        all_url.append(concurrent["url"])
        a["position"] = concurrent["position"]
        a["title"] = concurrent["title"]
        a["snippet"] = concurrent["snippet"]
        a["url"] = concurrent["url"]
        a["descr"] = concurrent["descr"]
        a["headings"] = concurrent["headings"]
        a["nb_words"] = concurrent["nb_words"]
        a["content"] = concurrent["content"]
        ## TODO : vérifier si le site est mieux classé que docaposte
        if (concurrent["position"] in positions[:top_concurrent+1] and 
            len(concurrents_data) < top_concurrent and 
            "docaposte" not in concurrent["url"]) :
            concurrents_data.append(a)
        if "docaposte" in concurrent["url"]:
            docaposte_data.append(a)
    
    ## Data on 'backlinks' key
    for _, b in results["datas"]["backlinks"].items():
        a = {}
        'title', 'headings', 'snippet', 'url', 'position'
        a["position"] = b["position"]
        a["title"] = b["title"]
        a["snippet"] = b["snippet"]
        a["url"] = b["url"]
        a["headings"] = b["headings"]
        a["nb_words"] = 0
        a["content"] = ""
        if ("docaposte" in b["url"]) :
            docaposte_data.append(a)
    
    
    obj = {'related_questions' : related_questions, 'other_related_questions' : other_related_questions, 
           'docaposte_data' : docaposte_data, 'concurrents_data' : concurrents_data, 'all_url' : all_url}
    return obj  




In [5]:
key_word = "Comment faire une signature électronique"
target_url = "https://www.docaposte.com/blog/article/comment-creer-signature-electronique-document"

In [6]:
%%time
results = get_semrank_result(key_word)

CPU times: user 57.5 ms, sys: 12 ms, total: 69.4 ms
Wall time: 13.3 s


In [7]:
import requests
from bs4 import BeautifulSoup

def get_hn_structure_and_content(url):
    response = requests.get(url)
    response.raise_for_status()  # Vérifie si la requête a réussi

    soup = BeautifulSoup(response.content, 'html.parser')
    # Extraction du titre de la page
    page_title = soup.title.string.strip() if soup.title else 'No title found'

    # Extraction de la description de la page
    meta_description = soup.find('meta', attrs={'name': 'description'})
    page_description = meta_description['content'].strip() if meta_description else 'No description found'

    # Extraction de la structure Hn et du contenu associé
    hn_structure = []
    current_section = None

    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div', 'ul']):
        if tag.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            if current_section:
                hn_structure.append(current_section)
            current_section = (tag.name.upper(), tag.get_text(strip=True), "")
        elif current_section:
            current_section = (current_section[0], current_section[1], current_section[2] + "\n" + tag.get_text(strip=True))

    if current_section:
        hn_structure.append(current_section)

    return page_title, page_description, hn_structure


page_title, page_description, hn_structure = get_hn_structure_and_content(target_url)

#print("Structure Hn et contenu:")
#for tag, title, content in hn_structure:
#    print(f"{tag}: {title}\nContenu:\n{content}\n")


In [8]:
parsed_res = parse_semrank_object(results, top_concurrent = 3)

In [9]:
keywords_list = [s.split(':')[0] for s in list(results['datas']['keywords_list'].values()) ]
#keywords_list

In [10]:
results

{'status': 'success',
 'datas': {'len_serper': 3,
  'len_contents': 9,
  'len_backlinks': 22,
  'len_backlinks_content': 10,
  'keywords_list': {'0': 'signature:36.55555555555556',
   '1': 'électronique:16.555555555555557',
   '2': 'document:10.222222222222221',
   '3': 'documents:7.555555555555555',
   '4': 'signer:7.444444444444445',
   '5': 'sélectionnez:6.222222222222222',
   '6': 'image:6.111111111111111',
   '7': 'signatures:5.888888888888889',
   '8': 'pdf:5.444444444444445',
   '9': 'nom:4.0',
   '10': 'mail:4.666666666666666',
   '11': 'cliquez:4.444444444444445',
   '12': 'processus:3.0',
   '13': 'ajouter:3.0',
   '14': 'niveau:3.888888888888889',
   '15': 'ligne:3.6666666666666665',
   '16': 'signé:3.5555555555555554',
   '17': 'créer:3.5555555555555554',
   '18': 'simple:3.4444444444444446',
   '19': 'signataire:3.4444444444444446',
   '20': 'étape:3.4444444444444446',
   '21': 'électroniques:3.4444444444444446',
   '22': 'informations:3.4444444444444446',
   '23': 'format

In [11]:
parsed_res

{'related_questions': ['Comment effectuer une signature électronique sur un document ?',
  'Comment me faire une signature électronique ?',
  'Comment se créer une signature électronique gratuite ?',
  'Comment mettre sa signature sur un document PDF ?'],
 'other_related_questions': ['Comment faire une signature électronique Word',
  'Signature électronique gratuite',
  'Comment faire une signature électronique sur un document PDF',
  'Comment faire une signature électronique gratuite',
  'Comment faire une signature électronique Outlook',
  'Signature électronique exemple',
  'Comment faire une signature électronique certifiée',
  'Comment faire une signature électronique sur iPhone'],
 'docaposte_data': [{'position': 23,
   'title': 'Comment obtenir une signature électronique qualifiée ? - Docaposte',
   'snippet': "La signature électronique qualifiée (QES) est le moyen le plus complet et sécurisé pour attester de l'identité du signataire d'un document. Elle ...",
   'url': 'https://

In [12]:
len(parsed_res["concurrents_data"])

3

In [13]:
################

### Embedding

In [14]:
## GPT4all
#gpt4all_embedding = GPT4AllEmbeddings()
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
gpt4all_embedding = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)

### LLM

In [15]:
# Mixtral
mistral_llm = instantiate_mixtral()
# OpenAI - GPT
gpt_llm = CustomGPT()

In [16]:
agent = get_agent(tools)
key_word = "Comment faire une signature électronique"
#res = agent({"input": key_word})

/Users/aba/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


### Get & Parse data

In [17]:
%%time
results = get_semrank_result(key_word)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
def parse_semrank_object(results, top_concurrent = 3):
    
    # 1. related questions
    related_questions = [i["question"] for _,i in results["datas"]["paa"].items()] 
    other_related_questions = [i["query"] for _,i in results["datas"]['related'].items()]
    
    # 2. Concurrent + docapposte data content => Title, snippet, Content, Hn structure
    list_concurr = results["datas"]['concurrents']
    concurrents_data = []
    positions = sorted([c["position"] for _, c in results["datas"]["concurrents"].items()])
    docaposte_data = []
    all_url = []
    
    ## Data on "concurrents" key
    for _, concurrent in results["datas"]["concurrents"].items():
        a = {}
        all_url.append(concurrent["url"])
        a["position"] = concurrent["position"]
        a["title"] = concurrent["title"]
        a["snippet"] = concurrent["snippet"]
        a["url"] = concurrent["url"]
        a["descr"] = concurrent["descr"]
        a["headings"] = concurrent["headings"]
        a["nb_words"] = concurrent["nb_words"]
        a["content"] = concurrent["content"]
        if (concurrent["position"] in positions[:top_concurrent+1] and 
            len(concurrents_data) < top_concurrent and 
            "docaposte" not in concurrent["url"]) :
            concurrents_data.append(a)
        if "docaposte" in concurrent["url"]:
            docaposte_data.append(a)
    
    ## Data on 'backlinks' key
    for _, b in results["datas"]["backlinks"].items():
        a = {}
        'title', 'headings', 'snippet', 'url', 'position'
        a["position"] = b["position"]
        a["title"] = b["title"]
        a["snippet"] = b["snippet"]
        a["url"] = b["url"]
        a["headings"] = b["headings"]
        a["nb_words"] = 0
        a["content"] = ""
        if ("docaposte" not in b["url"]) :
            concurrents_data.append(a)
        else:
            docaposte_data.append(a)
    
    
    obj = {'related_questions' : related_questions, 'other_related_questions' : other_related_questions, 
           'docaposte_data' : docaposte_data, 'concurrents_data' : concurrents_data, 'all_url' : all_url}
    return obj  

In [19]:
def parse_hn_from_dict(obj):
    
    # res = ['<' + h['type'] + '>' + h['text'] + '<' + h['type'] + '/>'  for _, h in obj.items() ]
    res = [ (h['type'], h['text'])  for _, h in obj.items() ]
    
    return res #'  \n'.join(res)


In [20]:
parsed_res = parse_semrank_object(results, top_concurrent = 3)

In [21]:
parse_hn_from_dict(parsed_res["concurrents_data"][0]["headings"])

[('h1', 'Comment faire une signature électronique'),
 ('h2', 'La signature électronique Comment ça marche'),
 ('h3', 'Fonctionnement'),
 ('h2', 'Valeur probante et validité'),
 ('h3', 'Identification multi-facteurs'),
 ('h3', 'Intégrité garantie'),
 ('h3', 'Dossier de preuves horodaté'),
 ('h2', 'Comment faire une signature électronique sur mes documents'),
 ('h3', 'De la consultation à l’approbation'),
 ('h3', 'Consultation de la demande'),
 ('h3', 'Validation de votre identité'),
 ('h3', 'Accord et signature'),
 ('h2', 'Faire une signature électronique avec l’application Yousign'),
 ('h3', 'Créer un compte sur l’application Yousign'),
 ('h3', 'Ajouter des contrats'),
 ('h3', 'Choisir des signataires'),
 ('h3', 'Notifications et signatures'),
 ('h2', 'Comment faire une signature numérique depuis mes outils'),
 ('h3', 'Une expérience fluide et sans friction'),
 ('h3', 'Une intégration dans vos outils'),
 ('h3', 'La simplicité pour votre équipe'),
 ('h3', 'Intégrez la signature électron

#### Proposition structure Hn

In [22]:
target_hn_structure = [(tag.lower(), title) for tag, title, content in hn_structure]
str(target_hn_structure)

"[('h1', 'Comment créer une signature électronique sur un document ?'), ('h2', 'Qu’est-ce qu’une signature électronique ?'), ('h2', 'Quels types de documents peut-on signer électroniquement ?'), ('h2', 'Quels sont les avantages de la signature électronique ?'), ('h2', 'Les différentes étapes pour créer une signature électronique'), ('h2', 'Les différents moyens de signer électroniquement un document'), ('h2', 'La gamme de signature électronique de Docaposte'), ('h2', 'Dans l’actualité')]"

In [23]:
n_concurrents = len(parsed_res["concurrents_data"])
concurrents_hn_structure =  [parse_hn_from_dict(parsed_res["concurrents_data"][i]["headings"]) for i in range(n_concurrents) ]
concurrents_hn_structure =  [[list(j) for j in i] for i in concurrents_hn_structure ]

"\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_hn_structure ])


'## Début exemple \n[[\'h1\', \'Comment faire une signature électronique\'], [\'h2\', \'La signature électronique Comment ça marche\'], [\'h3\', \'Fonctionnement\'], [\'h2\', \'Valeur probante et validité\'], [\'h3\', \'Identification multi-facteurs\'], [\'h3\', \'Intégrité garantie\'], [\'h3\', \'Dossier de preuves horodaté\'], [\'h2\', \'Comment faire une signature électronique sur mes documents\'], [\'h3\', \'De la consultation à l’approbation\'], [\'h3\', \'Consultation de la demande\'], [\'h3\', \'Validation de votre identité\'], [\'h3\', \'Accord et signature\'], [\'h2\', \'Faire une signature électronique avec l’application Yousign\'], [\'h3\', \'Créer un compte sur l’application Yousign\'], [\'h3\', \'Ajouter des contrats\'], [\'h3\', \'Choisir des signataires\'], [\'h3\', \'Notifications et signatures\'], [\'h2\', \'Comment faire une signature numérique depuis mes outils\'], [\'h3\', \'Une expérience fluide et sans friction\'], [\'h3\', \'Une intégration dans vos outils\'], [\

In [24]:
related_questions = parsed_res["related_questions"] + parsed_res["other_related_questions"]
prompt_structure = """
Vous êtes l'assistant d'un consultant SEO qui doit proposer un contenu pour le site web de son entreprise. Pour cela,
vous disposez de plusieurs exemples de structure Hn de pages bien référencées, parlant du sujet. Ces exemples doivent
vous servir pour proposer 3 nouvelles structures Hn pour aider le consultant SEO à rédiger son contenu. Vous utiliserez
aussi les questions que se posent souvent les internautes sur ce sujet pour adapter votre proposition de structure Hn.
Votre réponse doit être uniquement un dictionnaire de 3 listes de listes Python, qui donnent les 3 structures Hn à
utiliser pour le contenu. Votre réponse doit être comme dans les exemples ci-dessous, sans aucun commentaire.

### Entrées :

1. **Sujet de l'article à rédiger** :
{keyword}

2. **Autres questions similaires que se posent les internautes sur le sujet** :
{related_questions}

3. **Exemples structures Hn des pages les mieux référencées** :
{exemples_structures}

### Sortie attendue :
Un dictionnaire contenant trois propositions de structure Hn. Chaque proposition doit être sous forme de liste ayant des sous listes composés de 2 éléments sous ce format : ["h1", "le titre"].

Exemple de sortie :
{
    "proposition1" : [["h1", "Mon titre 1"], ["h2", "Mon sous-titre 1"], ["h2", "Mon sous-titre 2"], ["h1", "Mon titre 2"], ["h1", "Mon titre 3"]],
    "proposition2" : [["h1", "Mon titre 1"], ["h2", "Mon sous-titre 1"], ["h2", "Mon sous-titre 2"], ["h1", "Mon titre 2"], ["h1", "Mon titre 3"]],
    "proposition3" : [["h1", "Mon titre 1"], ["h2", "Mon sous-titre 1"], ["h2", "Mon sous-titre 2"], ["h1", "Mon titre 2"], ["h1", "Mon titre 3"]]
}

### Instructions :

1. Respecter strictement les formats demandés.
2. S'inspirer des exemples de structure fournis.
3. Utiliser des guillemets doubles pour tout texte dans le résultat.
""".replace("{exemples_structures}", "\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_hn_structure ]))\
.replace("{related_questions}", str("\n".join(related_questions)))\
.replace("{keyword}", str(key_word))


In [25]:
%%time
#Intention de recherche
def get_intention_recherche(keyword, llm):
    question = f"""Quels sont les principales informations clés que cherche à savoir l'internaute à travers la question suivante : \n {keyword}. 
    Donne une réponse synthétique avec les différents points."""
    intention: str = llm.invoke(question)

    return intention

get_intention_recherche(key_word, gpt_llm)

CPU times: user 55.7 ms, sys: 7.55 ms, total: 63.2 ms
Wall time: 11 s


"Lorsqu'un internaute se demande comment faire une signature électronique, il cherche généralement à obtenir des informations clés sur les aspects suivants :\n\n1. **Définition** : Comprendre ce qu'est une signature électronique et comment elle diffère d'une signature manuscrite.\n2. **Outils et logiciels** : Informations sur les logiciels ou services en ligne disponibles pour créer une signature électronique (Adobe Sign, DocuSign, etc.).\n3. **Processus** : Étapes détaillées pour créer une signature électronique :\n    - Télécharger le document à signer.\n    - Utiliser le logiciel ou la plateforme pour apposer la signature.\n    - Enregistrer et envoyer le document signé.\n4. **Validation légale** : Comprendre les aspects légaux et la validité juridique des signatures électroniques, en fonction des législations locales (eIDAS en Europe, ESIGN Act aux États-Unis, etc.).\n5. **Sécurité** : Informations sur la sécurité des signatures électroniques et comment assurer l'authenticité et l'

In [28]:
#prompt_structure
class StructureHn(BaseModel):
    structure: dict[str, list[list[str, str]]] = Field(description="Dictionnaire de 3 propositions de structure Hn. Ceux-ci sont au format de liste avec 2 éléments : le niveau (h1,h2, etc) et le titre")

class Content(BaseModel):
    content: dict[str, list[list[str, str, str]]] = Field(description="Dictionnaire de 3 propositions de contenu pour une structure Hn. Chaque proposition est au format de liste avec 3 éléments : le niveau (h1,h2, etc), le titre et le contenu")



def build_chain(llm, output_class):
    parser = PydanticOutputParser(pydantic_object=output_class)

    prompt = PromptTemplate(
        template="Answer the user query based on the information given.\n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | llm | parser
    
    return chain


chain_structure = build_chain(gpt_llm, StructureHn)
chain_content = build_chain(gpt_llm, Content)

In [29]:
%%time
res_struct_gpt = chain_structure.invoke({"query": prompt_structure})
res_struct_gpt

CPU times: user 86.4 ms, sys: 4.7 ms, total: 91.1 ms
Wall time: 8.07 s


StructureHn(structure={'proposition1': [['h1', 'Comment faire une signature électronique'], ['h2', "Qu'est-ce qu'une signature électronique ?"], ['h3', 'Définition et usages'], ['h3', 'Les différents types de signatures électroniques'], ['h3', 'Valeur légale et sécurité'], ['h2', 'Comment créer une signature électronique ?'], ['h3', 'Étape 1 : Choisir un outil de signature'], ['h3', 'Étape 2 : Créer un certificat de signature'], ['h3', 'Étape 3 : Signer un document'], ['h2', 'Comment utiliser la signature électronique sur différents formats de documents ?'], ['h3', 'Signer un PDF'], ['h3', 'Signer un document Word'], ['h3', 'Signer avec Outlook'], ['h2', 'Les avantages de la signature électronique'], ['h3', 'Rapidité et efficacité'], ['h3', 'Sécurité renforcée'], ['h3', 'Conformité réglementaire']], 'proposition2': [['h1', 'Comment faire ou créer une signature électronique'], ['h2', 'Préparation des documents pour signature électronique'], ['h3', 'Téléchargez vos documents'], ['h3', 'A

In [52]:
res_struct_gpt.structure

{'proposition1': [['h1', 'Introduction à la signature électronique'],
  ['h2', 'Fonctionnement de la signature électronique'],
  ['h2', 'Comment effectuer une signature électronique sur un document'],
  ['h2', 'Méthodes pour créer une signature électronique'],
  ['h2', 'Comment signer électroniquement un document PDF'],
  ['h1', 'Outils pour la signature électronique'],
  ['h2', 'Logiciels de signatures électroniques'],
  ['h2', 'Signature électronique gratuite'],
  ['h1', 'Questions fréquentes']],
 'proposition2': [['h1', 'Guide de la signature électronique'],
  ['h2', "Qu'est-ce que la signature électronique"],
  ['h2', 'Comment créer une signature électronique'],
  ['h2', 'Étapes pour faire une signature électronique sur un document'],
  ['h2', 'Faire une signature électronique Word'],
  ['h1', 'Les options de signatures électroniques'],
  ['h2', 'Signature électronique sur iPhone'],
  ['h2', 'Faire une signature électronique par mail'],
  ['h2', 'Faire une signature électronique Gm

In [30]:
%%time
res_struct_gpt = gpt_llm.invoke(prompt_structure)
res_struct_gpt

CPU times: user 44.5 ms, sys: 51.1 ms, total: 95.6 ms
Wall time: 13.8 s


'```python\n{\n    "proposition1": [\n        ["h1", "Comment faire une signature électronique"],\n        ["h2", "Qu\'est-ce qu\'une signature électronique ?"],\n        ["h2", "Pourquoi utiliser une signature électronique ?"],\n        ["h3", "Quels sont les avantages ?"],\n        ["h3", "Les inconvénients éventuels"],\n        ["h2", "Comment créer une signature électronique"],\n        ["h3", "Étape 1: Téléchargez le document"],\n        ["h3", "Étape 2: Ajoutez votre signature"],\n        ["h3", "Étape 3: Validez la signature"],\n        ["h2", "Solutions pour signer des documents"],\n        ["h3", "Comment signer un PDF"],\n        ["h3", "Comment signer un document Word"],\n        ["h3", "Autres outils de signature électronique"],\n        ["h2", "Questions fréquentes"],\n        ["h3", "Comment obtenir une signature électronique gratuite ?"],\n        ["h3", "Comment intégrer une signature électronique dans Outlook ?"],\n        ["h3", "Est-ce que la signature électronique e

In [32]:
%%time
res_struct_mistral = mistral_llm.invoke(prompt_structure)
res_struct_mistral

CPU times: user 37.9 ms, sys: 82.8 ms, total: 121 ms
Wall time: 2min 18s


'4. Utiliser des listes Python pour représenter les structures Hn.\n5. Utiliser des listes Python pour représenter les propositions de structure Hn.\n6. Utiliser des dictionnaires Python pour regrouper les propositions de structure Hn.\n7. Ne pas inclure de commentaires dans le résultat.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

In [33]:

import ast
import re
import json

def extract_tuples_from_text(text):
    s = text.find('[')
    e = text.rfind(']')
    try:
        return ast.literal_eval(text[s:e + 1])
    except:
        return []

# Exemple d'utilisation
text = "Le résultat est :  [('h1', 'Comment créer une signature électronique sur un document ?'), ('h2', 'Qu’est-ce qu’une signature électronique ?'), ('h3', 'Définition de la signature électronique'), ('h2', 'Quels types de documents peut-on signer électroniquement ?')]"
tuples_list = extract_tuples_from_text(res_struct_mistral)

print(tuples_list)

[]


In [34]:
def parse_text_to_dict(text):
    s = text.find('{')
    e = text.rfind('}')
    result_dict = {}
    try:
        # Nettoyer le texte et transformer les fausses échappements
        clean_text = text[s:e + 1].replace("\\'", "'")
        # Utiliser ast.literal_eval pour évaluer le texte en un dictionnaire Python
        result_dict = ast.literal_eval(clean_text)
    except:
        pass
    return result_dict
    

In [35]:
parse_text_to_dict(res_struct_gpt)

{'proposition1': [['h1', 'Comment faire une signature électronique'],
  ['h2', "Qu'est-ce qu'une signature électronique ?"],
  ['h2', 'Pourquoi utiliser une signature électronique ?'],
  ['h3', 'Quels sont les avantages ?'],
  ['h3', 'Les inconvénients éventuels'],
  ['h2', 'Comment créer une signature électronique'],
  ['h3', 'Étape 1: Téléchargez le document'],
  ['h3', 'Étape 2: Ajoutez votre signature'],
  ['h3', 'Étape 3: Validez la signature'],
  ['h2', 'Solutions pour signer des documents'],
  ['h3', 'Comment signer un PDF'],
  ['h3', 'Comment signer un document Word'],
  ['h3', 'Autres outils de signature électronique'],
  ['h2', 'Questions fréquentes'],
  ['h3', 'Comment obtenir une signature électronique gratuite ?'],
  ['h3', 'Comment intégrer une signature électronique dans Outlook ?'],
  ['h3', 'Est-ce que la signature électronique est légale ?']],
 'proposition2': [['h1',
   'Guide Complet pour Créer une Signature Électronique'],
  ['h2', 'Introduction à la signature élec

In [30]:
parse_text_to_dict(res_struct_mistral)

{'proposition1': [('h1', 'Comment faire une signature électronique'),
  ('h2', 'Qu’est-ce qu’une signature électronique ?'),
  ('h2', 'Pourquoi utiliser une signature électronique ?'),
  ('h2', 'Comment créer une signature électronique sur un document Word ?'),
  ('h2', 'Comment créer une signature électronique sur un document PDF ?'),
  ('h2', 'Comment envoyer une signature électronique par mail ?'),
  ('h2',
   'Comment faire une signature électronique sur un document gratuitement ?'),
  ('h2', 'Signature électronique certifiée : comment ça marche ?')],
 'proposition2': [('h1', 'Comment faire une signature électronique'),
  ('h2', 'Qu’est-ce qu’une signature électronique ?'),
  ('h2', 'Pourquoi utiliser une signature électronique ?'),
  ('h2', 'Comment créer une signature électronique sur un document Word ?'),
  ('h2', 'Comment créer une signature électronique sur un document PDF ?'),
  ('h2', 'Comment envoyer une signature électronique par mail ?'),
  ('h2', 'Signature électronique 

In [31]:
def hn_list_to_dict(obj)  -> dict:
    res = dict([(i, {obj[i][0], obj[i][1]}) for i in range(len(obj))])
    
    return res

In [56]:
import re

def html_to_markdown_with_levels(html_text):
    # Utiliser des expressions régulières pour trouver les balises Hn et leur contenu
    pattern = re.compile(r'<(h[1-6])>(.*?)<\/\1>', re.IGNORECASE)
    matches = pattern.findall(html_text)
    
    # Construire le texte Markdown
    markdown_lines = []
    for match in matches:
        tag, content = match
        level = int(tag[1])
        indent = ' ' * 4 * (level - 1)  # 4 espaces pour chaque niveau de tabulation
        markdown_lines.append(f"{indent}{tag.upper()}: {content.strip()}")
    
    # Joindre les lignes avec des sauts de ligne pour créer la structure Markdown finale
    markdown_text = '\n\n'.join(markdown_lines)
    return markdown_text

# Exemple d'utilisation
html_text = """<h1>Comment faire une signature électronique?</h1>
<h2>Qu'est-ce qu'une signature électronique?</h2>
<h2>Pourquoi utiliser une signature électronique?</h2>
<h3>Signature électronique : sécurité et confidentialité</h3>
<h3>Économie de temps et réduction des coûts</h3>
<h2>Comment créer une signature électronique sur différents supports</h2>
<h3>Signature électronique sur un document PDF</h3>
<h3>Signature électronique sur Word</h3>
<h3>Envoi d'une signature électronique par mail</h3>
<h2>Questions fréquemment posées</h2>"""

markdown_text = html_to_markdown_with_levels(html_text)
print(markdown_text)


H1: Comment faire une signature électronique?

    H2: Qu'est-ce qu'une signature électronique?

    H2: Pourquoi utiliser une signature électronique?

        H3: Signature électronique : sécurité et confidentialité

        H3: Économie de temps et réduction des coûts

    H2: Comment créer une signature électronique sur différents supports

        H3: Signature électronique sur un document PDF

        H3: Signature électronique sur Word

        H3: Envoi d'une signature électronique par mail

    H2: Questions fréquemment posées


In [55]:
a = ["""<h1>Comment faire une signature électronique?<h1/>\n\n<h2>Qu'est-ce qu'une signature électronique?<h2/>\n\n<h2>Pourquoi utiliser une signature électronique?<h2/>\n\n<h3>Signature électronique : sécurité et confidentialité<h3/>\n\n<h3>Économie de temps et réduction des coûts<h3/>\n\n<h2>Comment créer une signature électronique sur différents supports<h2/>\n\n<h3>Signature électronique sur un document PDF<h3/>\n\n<h3>Signature électronique sur Word<h3/>\n\n<h3>Envoi d'une signature électronique par mail<h3/>\n\n<h2>Questions fréquemment posées<h2/>""", 
"<h1>Guide de la signature électronique<h1/>\n\n<h2>Comprendre la signature électronique<h2/>\n\n<h3>Notions de base de la signature électronique<h3/>\n\n<h2>Processus de création d'une signature électronique<h2/>\n\n<h3>Étape par étape pour faire une signature électronique<h3/>\n\n<h2>Signature électronique et différents types de documents<h2/>\n\n<h3>Comment signer électroniquement un document PDF<h3/>\n\n<h3>Comment signer électroniquement un document Word<h3/>\n\n<h2>Exemples de signature électronique<h2/>\n\n<h2>FAQ sur la signature électronique<h2/>""", 
"<h1>Tout ce que vous devez savoir sur la signature électronique<h1/>\n\n<h2>C'est quoi une signature électronique<h2/>\n\n<h3>Définition de la signature électronique<h3/>\n\n<h2>Applications de la signature électronique<h2/>\n\n<h3>Signature électronique pour les e-mails<h3/>\n\n<h3>Signature électronique pour les documents PDF<h3/>\n\n<h3>Signature électronique pour les documents Word<h3/>\n\n<h2>Comment faire une signature électronique<h2/>\n\n<h3>Conseils et astuces pour créer une signature électronique<h3/>\n\n<h2>FAQs et mythes sur la signature électronique<h2/>"""]
t = (a[0]) # html_text
pattern = re.compile(r'<(h[1-6])>(.*?)<\/\1>', re.IGNORECASE)
matches = pattern.findall(t)
matches
#print(a[0])
#html_text
t

"<h1>Comment faire une signature électronique?<h1/>\n\n<h2>Qu'est-ce qu'une signature électronique?<h2/>\n\n<h2>Pourquoi utiliser une signature électronique?<h2/>\n\n<h3>Signature électronique : sécurité et confidentialité<h3/>\n\n<h3>Économie de temps et réduction des coûts<h3/>\n\n<h2>Comment créer une signature électronique sur différents supports<h2/>\n\n<h3>Signature électronique sur un document PDF<h3/>\n\n<h3>Signature électronique sur Word<h3/>\n\n<h3>Envoi d'une signature électronique par mail<h3/>\n\n<h2>Questions fréquemment posées<h2/>"

In [102]:
def generate_markdown_toc(hn_list):
    toc_lines = []
    # Dictionnaire pour suivre le compteur de chaque niveau de titre
    counters = {f'h{i}': 0 for i in range(1, 7)}
    previous_level = 0
    
    for level, title in hn_list:
        current_level = int(level[1])
        
        # Reset des compteurs des sous-niveaux si on change de niveau
        if current_level <= previous_level:
            for i in range(current_level + 1, 7):
                counters[f'h{i}'] = 0

        # Incrément du compteur du niveau actuel
        counters[level] += 1
        previous_level = current_level
        
        # Génération du numéro basé sur les compteurs
        num = ".".join(str(counters[f'h{i}']) for i in range(1, current_level + 1))
        
        # Ajout de l'entrée formatée dans la table des matières
        indent = '    ' * (current_level - 1)
        toc_lines.append(f"{indent}{num}. {title}")
    
    return "\n".join(toc_lines)

# Exemple d'utilisation
hn_list = [
    ('h1', 'Comment créer une signature électronique sur un document ?'),
    ('h2', 'Qu’est-ce qu’une signature électronique ?'),
    ('h3', 'Définition de la signature électronique'),
    ('h2', 'Quels types de documents peut-on signer électroniquement ?')
]

markdown_toc = generate_markdown_toc(hn_list)
print(markdown_toc)


1. Comment créer une signature électronique sur un document ?
    1.1. Qu’est-ce qu’une signature électronique ?
        1.1.1. Définition de la signature électronique
    1.2. Quels types de documents peut-on signer électroniquement ?


In [106]:
import json

def build_json_structure(hn_list):
    def add_node(level, text, structure):
        current_level = level
        if current_level not in structure:
            structure[current_level] = []
        structure[current_level].append({"text": text, "subtitles": {}})
        return structure[current_level][-1]["subtitles"]

    root = {}
    stack = []

    for level, text in hn_list:
        current_level = int(level[1])
        node = {"level": level, "text": text, "subtitles": {}}

        while stack and stack[-1][0] >= current_level:
            stack.pop()

        if stack:
            parent = stack[-1][1]
            if "subtitles" not in parent:
                parent["subtitles"] = {}
            add_node(level, text, parent["subtitles"])
        else:
            add_node(level, text, root)

        stack.append((current_level, node))

    return root

# Exemple d'utilisation
hn_list = [
    ('h1', 'Comment créer une signature électronique sur un document ?'),
    ('h2', 'Qu’est-ce qu’une signature électronique ?'),
    ('h3', 'Définition de la signature électronique'),
    ('h2', 'Quels types de documents peut-on signer électroniquement ?')
]

json_structure = build_json_structure(hn_list)
json_output = json.dumps(json_structure, ensure_ascii=False, indent=4)
print(json_output)


{
    "h1": [
        {
            "text": "Comment créer une signature électronique sur un document ?",
            "subtitles": {}
        }
    ]
}


#### Proposition titre

In [75]:
concurrents_titre =  [[('titre', parsed_res["concurrents_data"][i]["title"]),
                      ('description', parsed_res["concurrents_data"][i]["descr"])] for i in range(n_concurrents) ]
"\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_titre ])


'## Début exemple \n[(\'titre\', \'Comment créer une signature électronique en ligne | Acrobat Sign\'), (\'description\', \'Découvrez comment créer une signature électronique vérifiable pour vos documents avec Acrobat Sign. Testez l’application gratuitement et commencez à signer des documents de manière rapide et sécurisée.\')]\n## Fin exemple \n \n## Début exemple \n[(\'titre\', \'La signature électronique : un outil devenu incontournable - francenum.gouv.fr\'), (\'description\', "Le développement du télétravail et la nécessité de gagner en efficacité et en rapidité rendent la faculté de signer à distance désormais indispensable au bon fonctionnement de l\'entreprise. La signature électronique permet en effet de signer instantanément, où que l\'on soit, la quasi totalité des actes administratifs ou commerciaux. Suivez le guide !")]\n## Fin exemple \n \n## Début exemple \n[(\'titre\', \'Comment faire une signature électronique ? - YouTube\'), (\'description\', "Découvrez comment signer

In [80]:
target_titre = [('titre', page_title), 
                ('description', page_description)]

In [87]:
prompt_titre = """
Vous êtes l'assistant d'un SEO qui doit améliorer le contenu d'une page web d'une entreprise, afin qu'il soit mieux référencés pour les recherches
des internautes. Vous avez ci-dessous quelques exemples de titre et description de de pages qui sont mieux référencés. Vous avez également le 
titre et la description de la page web de l'entreprise, à améliorer. Vous devez vous inspirer des titres et description des pages mieux référencés 
pour proposer une amélioration du titre et de la description de la page de l'entreprise. Votre réponse doit être uniquement une liste de tuples 
Python, comme dans les exemples, sans aucun commentaire.

Voici les titres et descriptions des pages les mieux référencées : 
---
{exemples_titres}
---

Voici la structure Hn de la page à améliorer : 
---
{titre_initial}
---

Voici un exemple de réponse : 

[('titre', "Mon titre 1"), ("description", "Ma description")]
""".replace("{exemples_titres}", "\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_titre ]))\
.replace("{titre_initial}", str(target_titre))

In [88]:
prompt_titre

'\nVous êtes l\'assistant d\'un SEO qui doit améliorer le contenu d\'une page web d\'une entreprise, afin qu\'il soit mieux référencés pour les recherches\ndes internautes. Vous avez ci-dessous quelques exemples de titre et description de de pages qui sont mieux référencés. Vous avez également le \ntitre et la description de la page web de l\'entreprise, à améliorer. Vous devez vous inspirer des titres et description des pages mieux référencés \npour proposer une amélioration du titre et de la description de la page de l\'entreprise. Votre réponse doit être uniquement une liste de tuples \nPython, comme dans les exemples, sans aucun commentaire.\n\nVoici les titres et descriptions des pages les mieux référencées : \n---\n## Début exemple \n[(\'titre\', \'Comment créer une signature électronique en ligne | Acrobat Sign\'), (\'description\', \'Découvrez comment créer une signature électronique vérifiable pour vos documents avec Acrobat Sign. Testez l’application gratuitement et commencez

In [85]:
%%time
res_titre_gpt = gpt_llm.invoke(prompt_titre)
res_titre_gpt

CPU times: user 48.9 ms, sys: 20.8 ms, total: 69.7 ms
Wall time: 10.1 s


"[('titre', 'Créer une signature électronique en ligne avec Docaposte | Signature électronique pour documents'), ('description', 'Découvrir avec Docaposte comment créer une signature électronique fiable et sécurisée. Facilitez vos processus administratifs et commerciaux avec notre système de signature en ligne, un outil rapide, efficace et devenu incontournable pour signer instantanément, où que vous soyez.')]"

In [64]:
%%time
res_titre_mistral = mistral_llm.invoke(prompt_titre)
#res_titre_mistral

NameError: name 'prompt_titre' is not defined

#### Proposition contenu 

In [56]:
concurrents_content = [parsed_res["concurrents_data"][i]["content"] for i
                                            in range(n_concurrents)]
len(concurrents_content)

int(np.mean([len(x) for x in concurrents_content]))

3052

In [58]:
selected_structure = res_struct_gpt.structure['proposition1']
selected_structure

[['h1', 'Introduction à la signature électronique'],
 ['h2', 'Fonctionnement de la signature électronique'],
 ['h2', 'Comment effectuer une signature électronique sur un document'],
 ['h2', 'Méthodes pour créer une signature électronique'],
 ['h2', 'Comment signer électroniquement un document PDF'],
 ['h1', 'Outils pour la signature électronique'],
 ['h2', 'Logiciels de signatures électroniques'],
 ['h2', 'Signature électronique gratuite'],
 ['h1', 'Questions fréquentes']]

In [78]:
concurrents_content = [parsed_res["concurrents_data"][i]["content"] for i
                                            in range(10)]
len_content = int(np.mean([len(x) for x in concurrents_content]))
prompt_content = """
Vous êtes l'assistant d'un consultant SEO qui doit proposer un article sur un sujet pour le site web de son entreprise.
Pour cela, vous disposez de plusieurs exemples d'articles et d'une structure Hn à respecter, parlant du sujet.
Ces exemples doivent vous servir pour proposer 3 versions d'article pour aider le consultant SEO. Chaque partie de la
structure Hn devrait être remplie avec un contenu adapté. Vous pouvez vous baser également sur des connaissances générales
pour remplir le contenu et pas seulement les exemples de contenu fournis.
La structure Hn proposée est sous format de liste de tuples avec le niveau du titre (h1, h2, etc) et le texte du titre.
La réponse doit comporter environ {len_content} caractères. L
Votre réponse doit être uniquement un dictionnaire contenant 3 éléments et correspondant aux 3 contenus générés.
Le format de la réponse doit être comme dans les exemples ci-dessous, sans aucun commentaire.
Vous avez en entrée : le sujet pour le contenu à rédiger, les exemples de contenu, la structure Hn à respecter.

Voici le sujet pour le contenu à rédiger :
---
{keyword}
---

Voici les exemples de contenus :
---
{exemples_contenus}
---

Voici la structure Hn à respecter :
---
{structure_hn}
---

Voici le format de la réponse à donner:
{
    "proposition1" : "Comment faire une signature electronique : mode d'emploi. \n Pour faire une signature, il faut 3 élements.",
    "proposition2" : "La signature electronique en détail. \n Il existe plusieurs moyen pour faire une signature.",
    "proposition3" : "Faire une signature electronique. \n La signature électronique devient la norme."
}
""".replace("{exemples_contenus}", "\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_content ]))\
.replace("{structure_hn}", str(selected_structure))\
.replace("{keyword}", str(key_word))


In [59]:
def generate_prompt_content(subject, structure_hn, examples, len_content):
    prompt = f"""
    Vous êtes un assistant et vous aidez un consultant SEO pour rédiger des contenus d'articles. 
    Je vais vous fournir une structure Hn à respecter pour rédiger l'article. 
    Cette structure est sous forme d'une liste de sous-listes du type ["h1", "titre de la section"]. 
    Je vais également vous donner plusieurs exemples d'articles pour vous inspirer. 
    Vous devez générer trois propositions de contenu d'article en respectant la structure Hn fournie.

    ### Entrées :

    1. **Sujet de l'article à rédiger** :
       {subject}

    2. **Structure Hn à respecter** :
       {structure_hn}

    3. **Exemples d'articles** :
       {examples}

    ### Sortie attendue :

    Un dictionnaire contenant trois propositions de contenu d'article, chaque proposition respectant la structure Hn 
    fournie et mis sous format de liste. Ces listes doivent contenir des sous-listes ayant trois éléments qui sont le 
    niveau du titre (h1, h2, etc), le texte du titre et le texte du contenu généré. Le format de ces sous-listes est 
    le suivant : ["h1", "le titre", "contenu généré"].

    Exemple de sortie :
    {{
        "proposition1": [
            ["h1", "Titre principal", "Contenu généré pour le titre principal"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"],
            ["h3", "Sous-sous-titre", "Contenu généré pour le sous-sous-titre"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"]
        ],
        "proposition2": [
            ["h1", "Titre principal", "Contenu généré pour le titre principal"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"],
            ["h3", "Sous-sous-titre", "Contenu généré pour le sous-sous-titre"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"]
        ],
        "proposition3": [
            ["h1", "Titre principal", "Contenu généré pour le titre principal"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"],
            ["h3", "Sous-sous-titre", "Contenu généré pour le sous-sous-titre"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"]
        ]
    }}

    ### Instructions :

    1. Respecter strictement la structure Hn fournie pour chaque proposition.
    2. S'inspirer des exemples d'articles fournis pour le ton et le style.
    3. Générer un contenu original et informatif pour chaque section de chaque proposition.
    4. Chaque proposition de contenu doit avoir environ {len_content} caractères de texte.
    5. Utiliser des guillemets doubles pour tout texte dans le résultat.
    """
    return prompt

len_content = int(np.mean([len(x) for x in concurrents_content]))

prompt_content2 = generate_prompt_content(key_word, selected_structure, 
                                          "\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_content ]),
                                         len_content)
len(prompt_content2)

37038

In [62]:
%%time
res_content_gpt = chain_content.invoke({"query": prompt_content2})
res_content_gpt

CPU times: user 57 ms, sys: 52.3 ms, total: 109 ms
Wall time: 2min 10s


Content(content={'proposition1': [['h1', 'Introduction à la signature électronique', "La signature électronique est un outil moderne qui facilite les échanges professionnels. Sa capacité à signer instantanément n'importe quel document est devenue essentielle pour les entreprises de toutes tailles. Mais comment fonctionne-t-elle et comment peut-on la créer ?"], ['h2', 'Fonctionnement de la signature électronique', "Au coeur de son système, la signature électronique utilise une technologie de cryptographie pour garantir l'authenticité du signataire et l'intégrité du document. Le but principal est donc de s'assurer que le document n'a pas été modifié une fois signé et que l'auteur du document est bien celui qu'il prétend être."], ['h2', 'Comment effectuer une signature électronique sur un document', "Plusieurs étapes sont nécessaires pour créer une signature électronique. Après avoir reçu une invitation par email pour signer un document, l'utilisateur doit confirmer son consentement et in

In [69]:
a, b, c = res_content_gpt.content['proposition1'][0]
c

"La signature électronique est un outil moderne qui facilite les échanges professionnels. Sa capacité à signer instantanément n'importe quel document est devenue essentielle pour les entreprises de toutes tailles. Mais comment fonctionne-t-elle et comment peut-on la créer ?"

In [83]:
%%time
res_content_gpt = gpt_llm.invoke(prompt_content)
res_content_gpt

CPU times: user 47.7 ms, sys: 11.1 ms, total: 58.8 ms
Wall time: 1min 47s


'{\n    "proposition1" : {\n        "h1": "Comprendre la signature électronique",\n        "h2": ["La signature électronique est un mécanisme permettant de garantir l\'intégrité d\'un document électronique, d\'authentifier son auteur et de rapporter la preuve du consentement. C\'est un moyen de signer instantanément, où que l\'on soit, la quasi totalité des actes administratifs ou commerciaux.", \n                "Il existe plusieurs types de signatures électroniques : simple, avancée ou qualifiée, qui varient en termes de niveau de fiabilité et de sécurité.", \n                "La signature électronique offre de nombreux avantages : elle facilite l\'envoi et l\'échange des documents, accélère la procédure de signature et sécurise les données. Cependant, tous les types de signatures électroniques ne disposent pas du même degré de force probante."],\n\n        "h1": "Comment réaliser une signature électronique",\n        "h2": ["Pour faire une signature électronique, il faut d\'abord té

In [70]:
%%time
res_content_mistral = mistral_llm.invoke(prompt_content)
res_content_mistral

CPU times: user 12.1 ms, sys: 10.7 ms, total: 22.8 ms
Wall time: 49.7 s


'\nVoici la réponse :\n\n{\n    "proposition1" : "Comment faire une signature électronique : mode d\'emploi. <br> Pour faire une signature électronique, il faut trois éléments : un document à signer, un outil de signature électronique et un signataire.<br><br> **Étape 1 : Téléchargez vos documents**<br> Téléchargez vos documents au format PDF, Word, Excel ou tout autre format à partir de votre ordinateur, de votre smartphone ou d\'un site de partage de fichiers.<br><br> **Étape 2 : Ajoutez les signataires**<br> Indiquez le nom et l\'adresse e-mail de vos signataires et précisez l\'ordre dans lequel ils doivent recevoir le document à signer.<br><br> **Étape 3 : Disposez des champs et envoyez**<br> Placez les champs de signature électronique par glisser-déposer à l\'endroit du fichier où vous avez besoin d\'une signature, d\'un paraphe ou de dates.<br><br> Cliquez ensuite sur Envoyer. DocuSign envoie par un mail un lien à chaque destinataire sur lequel ils doivent cliquer pour accéder au

In [81]:
%%time
res_content_gpt2 = gpt_llm.invoke(prompt_content2)
res_content_gpt2

CPU times: user 48.8 ms, sys: 70.7 ms, total: 119 ms
Wall time: 2min 24s


'{\n    "proposition1": [\n        (\'h1\', \'Comprendre la signature électronique\', \n         "Des contrats d\'entreprise aux formulaires en ligne, la signature électronique est de plus en plus couramment utilisée pour confirmer son accord. Cette méthode numérique, à la fois pratique et valable juridiquement, offre une grande flexibilité."),\n        (\'h2\', "Qu\'est-ce qu\'une signature électronique ?",\n         "Il s\'agit de l\'équivalent numérique de votre signature manuscrite. Elle est utilisée pour approuver des informations ou des documents dans un format électronique. Cela peut être aussi simple qu\'un nom tapé, une image numérisée ou une marque unique créée via un outil numérique."),\n        (\'h2\', \'Les différents types de signature électronique\',\n         "Il existe plusieurs types, allant de la signature simple à la signature qualifiée. Le choix dépend de votre besoin en matière de sécurité et de validation. Le niveau de sécurité et d\'authentification varie en fo

In [85]:
def parse_text_to_dict(text: str) -> dict:
    s = text.find('{')
    e = text.rfind('}')
    result_dict = {}
    try:
        # Nettoyer le texte et transformer les fausses échappements
        clean_text = text[s:e + 1].replace("\\'", "'")
        # Utiliser ast.literal_eval pour évaluer le texte en un dictionnaire Python
        result_dict = ast.literal_eval(clean_text)
    except:
        pass
    return result_dict

In [86]:
parse_text_to_dict(res_content_gpt2)

{'proposition1': [('h1',
   'Comprendre la signature électronique',
   "Des contrats d'entreprise aux formulaires en ligne, la signature électronique est de plus en plus couramment utilisée pour confirmer son accord. Cette méthode numérique, à la fois pratique et valable juridiquement, offre une grande flexibilité."),
  ('h2',
   "Qu'est-ce qu'une signature électronique ?",
   "Il s'agit de l'équivalent numérique de votre signature manuscrite. Elle est utilisée pour approuver des informations ou des documents dans un format électronique. Cela peut être aussi simple qu'un nom tapé, une image numérisée ou une marque unique créée via un outil numérique."),
  ('h2',
   'Les différents types de signature électronique',
   "Il existe plusieurs types, allant de la signature simple à la signature qualifiée. Le choix dépend de votre besoin en matière de sécurité et de validation. Le niveau de sécurité et d'authentification varie en fonction du type."),
  ('h2',
   'Avantages et inconvénients de

In [84]:
%%time
res_content_mistral2 = mistral_llm.invoke(prompt_content2)
res_content_mistral2

CPU times: user 17.3 ms, sys: 83.4 ms, total: 101 ms
Wall time: 3min 30s


"5. Utiliser les mots-clés fournis dans les titres.\n    6. Ne pas inclure de contenu en dehors des titres et du contenu généré.\n    7. Ne pas inclure de liens externes.\n    8. Ne pas inclure d'images.\n    9. Ne pas inclure de code.\n    10. Ne pas inclure de tableaux.\n    11. Ne pas inclure de listes à puces ou numérotées.\n    12. Ne pas inclure de citations.\n    13. Ne pas inclure de notes de bas de page.\n    14. Ne pas inclure d'annotations.\n    15. Ne pas inclure de métadonnées.\n    16. Ne pas inclure de balises HTML.\n    17. Ne pas inclure de balises de formatage.\n    18. Ne pas inclure de balises de style.\n    19. Ne pas inclure de balises de script.\n    20. Ne pas inclure de balises méta.\n    21. Ne pas inclure de balises d'en-tête.\n    22. Ne pas inclure de balises de corps.\n    23. Ne pas inclure de balises de pied de page.\n    24. Ne pas inclure de balises de section.\n    25. Ne pas inclure de balises de div.\n    26. Ne pas inclure de balises d'article.\n  

#### Proposition résumé 